### bd econ CPS extract

bd_CPS_benchmark.ipynb

January 6, 2018

Contact: Brian Dew, @bd_econ

Requires: `cpsYYYY.ft` files for each year. The bd CPS files are generated bd_CPS_reader.ipynb

-----

See [readme](https://github.com/bdecon/econ_data/tree/master/bd_CPS) for documentation.

In [1]:
import pandas as pd
import wquantiles
import os

os.chdir('/home/brian/Documents/CPS/data/clean/')

#### Benchmark 1

In October 1999, how many people were unemployed because of losing a job?

BLS: LNU03023621: 2,162,000

In [2]:
(pd.read_feather('cps1999.ft')
   .query('HRMONTH==10 and UNEMPTYPE == "Job Loser"')
   ['PWCMPWGT']).sum()

2161502.5

#### Benchmark 2

In February 2007, what share of age 25-54 women were employed?

BLS: LNU02300062: 72.6

In [3]:
df = (pd.read_feather('cps2007.ft')
        .query('HRMONTH==2 and 25 <= AGE <= 54 and FEMALE==1')
        .groupby('EMP')
        .PWCMPWGT.sum())

df[1] / df.sum()

0.72602457

#### Benchmark 3

In May 2014, how many people have more than one job?

BLS: LNU02026619: 7,305,000

In [4]:
(pd.read_feather('cps2014.ft')
   .query('HRMONTH==5 and MJH==1')
   .PWCMPWGT).sum()

7304317.5

#### Benchmark 4

In 2017 Q1, what were the nominal usual weekly earnings?

BLS: $857

In [5]:
df = (pd.read_feather('cps2017.ft')
        .query('HRMONTH < 4 and WKWAGE > 0 and PRFTLF==1'))

wquantiles.median(df['WKWAGE'], df['PWORWGT'])

865.0